Creating baseline model EGARCH for ETH

In [1]:
import warnings
import numpy as np
import pandas as pd
from statsmodels.tsa.stattools import adfuller, kpss
from arch.univariate import EGARCH, ZeroMean, StudentsT, Normal
from statsmodels.tools.sm_exceptions import InterpolationWarning

Helper Function 

In [2]:
#need to import this for rmse and qlike
#import numpy as np

def rmse(y_true, y_pred):
    return float(np.sqrt(np.mean((y_true - y_pred) ** 2)))


def qlike(y2, var_fc, eps=1e-12):
    ratio = (y2 + eps) / (var_fc + eps)
    return float(np.mean(ratio - np.log(ratio) - 1.0))


def clean_series(x, name="series"):
    x = pd.to_numeric(x, errors="coerce").astype(float)
    x = x.replace([np.inf, -np.inf], np.nan).dropna()
    if not x.index.is_monotonic_increasing:
        x = x.sort_index()
    return x


def scale_series(x, factor=100.0):
    if factor is None or factor == 1.0:
        return x, 1.0
    return x * factor, factor

Loading dataset 

In [3]:
df = pd.read_csv("../Data/eth_final_df.csv")

df['timestamp'] = pd.to_datetime(df['timestamp'], utc=True)
df = df.set_index('timestamp')
df = df.sort_index()
df = df[~df.index.duplicated(keep='last')]
df = df.asfreq('h')  # make sure it's hourly


r = df['hourly_return']

print("Index dtype:", df.index.dtype)
print("Columns:", df.columns)
print("Data shape:", df.shape)
print(f"\nUsing {len(r)} observations for EGARCH modelling.")

Index dtype: datetime64[ns, UTC]
Columns: Index(['RV_MA_1hr', 'RV_MA_3hr', 'RV_MA_12hr', 'vol_future',
       'active_sending_addresses', 'active_receiving_addresses',
       'exchange_withdrawing_count', 'transaction_count', 'fail_rate_percent',
       'open', 'low', 'high', 'close', 'hourly_return',
       'onchain_volume_usd_log', 'avg_gas_fee_usd_log',
       'avg_priority_fee_usd_log', 'staking_inflow_log',
       'exchange_depositing_count_log', 'exchange_netflow_usd_log',
       'whale_net_usd', 'whale_net_usd_24h', 'whale_burst_flag',
       'etow_usd_log', 'etow_coins_log', 'whale_txn_count_log', 'wtoe_usd_log',
       'wtoe_coins_log', 'btc_to_eth_spill'],
      dtype='object')
Data shape: (8208, 29)

Using 8208 observations for EGARCH modelling.


Defining stationary tests 

In [4]:
def adf_kpss(x: pd.Series):
    x = x.dropna()
    adf_res = adfuller(x, autolag="AIC")
    kpss_level = kpss(x, regression="c", nlags="auto")
    kpss_trend = kpss(x, regression="ct", nlags="auto")
    return {
        "ADF": {"stat": adf_res[0], "p": adf_res[1]},
        "KPSS_level": {"stat": kpss_level[0], "p": kpss_level[1]},
        "KPSS_trend": {"stat": kpss_trend[0], "p": kpss_trend[1]},
    }

print("\n=== Stationarity Tests ===")
print(adf_kpss(r))


=== Stationarity Tests ===
{'ADF': {'stat': np.float64(-18.646556209260286), 'p': np.float64(2.052218387867681e-30)}, 'KPSS_level': {'stat': np.float64(0.1810942409319947), 'p': np.float64(0.1)}, 'KPSS_trend': {'stat': np.float64(0.14541966082436722), 'p': np.float64(0.051074702177097725)}}


/var/folders/ql/bmfws9h55z1g8ckt55h4p8jr0000gn/T/ipykernel_6866/463190570.py:4: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is greater than the p-value returned.

  kpss_level = kpss(x, regression="c", nlags="auto")


Use table to show at least weak stationarity 

In [5]:
def stationarity_table(x, alpha=0.05):
    x = pd.to_numeric(x, errors="coerce").astype(float).dropna()

    adf_stat, adf_p, adf_lags, adf_nobs, adf_crit, *_ = adfuller(x, autolag="AIC")

    with warnings.catch_warnings():
        warnings.filterwarnings("ignore", category=InterpolationWarning)
        kpss_level_stat, kpss_level_p, kpss_level_lags, kpss_level_crit = kpss(x, regression="c", nlags="auto")
        kpss_trend_stat, kpss_trend_p, kpss_trend_lags, kpss_trend_crit = kpss(x, regression="ct", nlags="auto")

    rows = [
        {
            "Test": "ADF",
            "Statistic": adf_stat,
            "p-value": adf_p,
            "H0": "Unit root (non-stationary)",
            f"Decision @ {int(alpha*100)}%": "Stationary (reject H0)" if adf_p < alpha else "Non-stationary (fail to reject)",
            "Lags used": adf_lags,
            "Obs used": adf_nobs,
        },
        {
            "Test": "KPSS (level)",
            "Statistic": kpss_level_stat,
            "p-value": kpss_level_p,
            "H0": "Level-stationary",
            f"Decision @ {int(alpha*100)}%": "Non-stationary (reject H0)" if kpss_level_p < alpha else "Stationary (fail to reject)",
            "Lags used": kpss_level_lags,
            "Obs used": len(x),
        },
        {
            "Test": "KPSS (trend)",
            "Statistic": kpss_trend_stat,
            "p-value": kpss_trend_p,
            "H0": "Trend-stationary",
            f"Decision @ {int(alpha*100)}%": "Non-stationary (reject H0)" if kpss_trend_p < alpha else "Stationary (fail to reject)",
            "Lags used": kpss_trend_lags,
            "Obs used": len(x),
        },
    ]

    df_out = pd.DataFrame(rows)
    return df_out

    num_cols = ["Statistic", "p-value"]
    df_out[num_cols] = df_out[num_cols].applymap(lambda v: np.nan if pd.isna(v) else float(v))
    return df_out

tbl = stationarity_table(r, alpha=0.05)
tbl

,Test,Statistic,p-value,H0,Decision @ 5%,Lags used,Obs used
0,ADF,-18.646556,2.052218e-30,Unit root (non-stationary),Stationary (reject H0),25,8182
1,KPSS (level),0.181094,1.000000e-01,Level-stationary,Stationary (fail to reject),14,8208
2,KPSS (trend),0.145420,5.107470e-02,Trend-stationary,Stationary (fail to reject),14,8208


Train+Validation/Test Split (85/15)

In [6]:
use_validation = True 
n = len(r)
if use_validation:
    n_train = int(0.70 * n)
    n_val   = int(0.15 * n)
    train_proto = r.iloc[:n_train]
    val_proto   = r.iloc[n_train:n_train+n_val]
    test_proto  = r.iloc[n_train+n_val:]
else:
    n_train = int(0.85 * n)
    train_proto = r.iloc[:n_train]
    val_proto   = None
    test_proto  = r.iloc[n_train:]

PURGE_HOURS = 24
N_SPLITS = 5 
if len(test_proto) == 0:
    raise ValueError("Empty test set after temporal split.")
boundary = test_proto.index.min()
purged_start = boundary + pd.Timedelta(hours=PURGE_HOURS)

train_val = r.loc[: purged_start - pd.Timedelta(hours=1)]
test      = r.loc[purged_start:]

print(f"Train/Val for CV: {len(train_val)} obs  |  Test (after 24h purge): {len(test)} obs")

#picking fixed sizes to fit in train_val
N_SPLITS    = 5
TEST_SIZE   = 24 * 5      
n           = len(train_val)

# Pick TRAIN_SIZE to fit the feasibility condition:
TRAIN_SIZE = n - (PURGE_HOURS + N_SPLITS * TEST_SIZE)

print(f"Rolling CV plan → train={TRAIN_SIZE}, test={TEST_SIZE}, purge={PURGE_HOURS}, folds={N_SPLITS} "
      f"(train_val length={n})")

Train/Val for CV: 7000 obs  |  Test (after 24h purge): 1208 obs
Rolling CV plan → train=6376, test=120, purge=24, folds=5 (train_val length=7000)


Rolling Window = 5 Splits 

In [11]:
def rolling_split_fixed(x, *, train_size, test_size, purge=24, n_splits=None):
    n = len(x)
    step = test_size
    start_train = 0
    k = 0

    while True:
        end_train   = start_train + train_size               
        start_test  = end_train + purge
        end_test    = start_test + test_size                  

        if end_test > n:                                     
            break
        yield (np.arange(start_train, end_train),
               np.arange(start_test,  end_test))

        k += 1
        if (n_splits is not None) and (k >= n_splits):
            break
        
        start_train += step

Fit EGARCH

In [8]:
def egarch_path_refit(train_ret, test_ret, dist="t", scale=100.0):
    tr = clean_series(train_ret, "train_ret")
    te = clean_series(test_ret, "test_ret")

    tr_s, s = scale_series(tr, scale)
    te_s, _ = scale_series(te, scale)

    all_s = pd.concat([tr_s, te_s])
    n_tr, n_te = len(tr_s), len(te_s)

    var_fc_s = np.empty(n_te)

    for j in range(n_te):
        y_fit = all_s.iloc[: n_tr + j]
        model = ZeroMean(y_fit)
        model.volatility = EGARCH(p=1, o=1, q=1)
        model.distribution = StudentsT() if dist == "t" else Normal()
        res = model.fit(disp="off")

        fcast = res.forecast(horizon=1, reindex=False)
        var_fc_s[j] = float(fcast.variance.values[-1, 0])

    var_fc = var_fc_s / (s ** 2)
    r2 = te.values ** 2

    return {
        "var_fc": var_fc,
        "r2": r2,
        "rmse": rmse(r2, var_fc),
        "qlike": qlike(r2, var_fc),
        "n_train": len(tr),
        "n_test": len(te),
    }

Add RMSE Tuning 

In [9]:
candidates = [
    ("t", 100.0),
    ("normal", 100.0),
    ("t", 1000.0),
    ("normal", 1000.0),
]

tune_results = []
for dist, scale in candidates:
    o = egarch_path_refit(train_val, test, dist=dist, scale=scale)
    tune_results.append((dist, scale, o["rmse"]))

#rank by lowest RMSE 
tune_results = sorted(tune_results, key=lambda x: x[2])
best_dist, best_scale, best_rmse = tune_results[0]

print("Best EGARCH tuning by RMSE")
print(f"  Distribution: {best_dist}")
print(f"  Scale:        {best_scale}")
print(f"  RMSE:         {best_rmse:.6f}")


Best EGARCH tuning by RMSE
  Distribution: t
  Scale:        100.0
  RMSE:         0.000133


Cross Validation (5 splits)

In [12]:
print("\n=== Rolling CV ===")
cv_scores = []

for i, (tri, tei) in enumerate(
        rolling_split_fixed(train_val,
                            train_size=TRAIN_SIZE,
                            test_size=TEST_SIZE,
                            purge=PURGE_HOURS,
                            n_splits=N_SPLITS), 1):
    tr_i = train_val.iloc[tri]
    te_i = train_val.iloc[tei]

    out = egarch_path_refit(
        tr_i, te_i,
        dist=best_dist if 'best_dist' in globals() else "t",
        scale=best_scale if 'best_scale' in globals() else 100.0
    )
    cv_scores.append((out["rmse"], out["qlike"]))
    print(f"Fold {i}: RMSE={out['rmse']:.6e} | QLIKE={out['qlike']:.6f} "
          f"| Train={out['n_train']} Test={out['n_test']}")

if cv_scores:
    mean_rmse  = float(np.mean([s[0] for s in cv_scores]))
    mean_qlike = float(np.mean([s[1] for s in cv_scores]))
    print(f"\nMean CV RMSE={mean_rmse:.6e} | Mean CV QLIKE={mean_qlike:.6f}")
else:
    print("No valid CV folds.")


=== Rolling CV ===
Fold 1: RMSE=7.989145e-05 | QLIKE=1.420418 | Train=6376 Test=120
Fold 2: RMSE=9.802757e-05 | QLIKE=1.738594 | Train=6376 Test=120
Fold 3: RMSE=1.153271e-04 | QLIKE=1.812964 | Train=6376 Test=120
Fold 4: RMSE=8.584808e-05 | QLIKE=1.270342 | Train=6376 Test=120
Fold 5: RMSE=4.865892e-04 | QLIKE=2.177654 | Train=6376 Test=120

Mean CV RMSE=1.731367e-04 | Mean CV QLIKE=1.683994


Final Evaluation 

In [13]:
final_train = train_val 
final_out = egarch_path_refit(final_train, test, dist="t", scale=100.0)
print("\n=== Final Test Results ===")
print(f"Test RMSE={final_out['rmse']:.6e} | QLIKE={final_out['qlike']:.6f}")


=== Final Test Results ===
Test RMSE=1.325891e-04 | QLIKE=1.913727


Check on why RMSE is so low - due to scaling since returns are of small magnitude 

In [14]:
print("Scale checks:")
print(f"mean|r|     = {np.mean(np.abs(r)):.3e}")
print(f"mean r^2    = {np.mean(r**2):.3e}")
print(f"std(r)^2    = {np.std(r)**2:.3e}")  


print("\nMetrics (scientific):")
print(f"RMSE       = {final_out['rmse']:.3e}")   
print(f"QLIKE      = {final_out['qlike']:.6f}")


vf = final_out['var_fc']
print("\nForecast variance summary:")
print(f"min={vf.min():.3e}  p50={np.median(vf):.3e}  mean={vf.mean():.3e}  max={vf.max():.3e}")

Scale checks:
mean|r|     = 4.963e-03
mean r^2    = 5.731e-05
std(r)^2    = 5.731e-05

Metrics (scientific):
RMSE       = 1.326e-04
QLIKE      = 1.913727

Forecast variance summary:
min=9.852e-06  p50=3.511e-05  mean=4.586e-05  max=4.232e-04


Save CSV for ETH EGARCH Prediction

In [15]:
pred_vol_future = np.sqrt(np.maximum(final_out["var_fc"], 0.0))
vol_future_df = pd.DataFrame(
    {
        "pred_vol_future": pred_vol_future,
    },
    index=test.index,
)


print("=== EGARCH Volatility Forecast Results ===")
display(vol_future_df.head(20))  


output_path = "../Results/eth_egarch_prediction.csv"
vol_future_df.to_csv(output_path, index_label="timestamp")

print(f"\nSaved EGARCH results to: {output_path}")

=== EGARCH Volatility Forecast Results ===


,pred_vol_future
timestamp,
2025-08-23 16:00:00+00:00,0.006138
2025-08-23 17:00:00+00:00,0.006046
2025-08-23 18:00:00+00:00,0.005640
2025-08-23 19:00:00+00:00,0.005354
2025-08-23 20:00:00+00:00,0.005124
2025-08-23 21:00:00+00:00,0.004994
2025-08-23 22:00:00+00:00,0.004719
2025-08-23 23:00:00+00:00,0.004878
2025-08-24 00:00:00+00:00,0.004710



Saved EGARCH results to: ../Results/eth_egarch_prediction.csv
